In [1]:
import os
print(os.getcwd())  # Shows where Python thinks you are

c:\Users\ishaa\OneDrive\Documents\Ishaan\AIML\NN\MNIST_784x16x16x10


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

In [3]:
data = pd.read_csv(r"C:\Users\ishaa\OneDrive\Documents\Ishaan\AIML\NN\MNIST_784x10x10\train.csv\train.csv")  # Note the 'r' prefix and backslashes

In [4]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
m_train = X_train.shape

In [5]:
Y_train

array([2, 1, 6, ..., 4, 0, 0], dtype=int64)

In [6]:
#FWD PROP
def init_params():
    W1 = np.random.rand(48, 784) - 0.5 
    b1 = np.random.rand(48, 1) - 0.5
    W2 = np.random.rand(48, 48) - 0.5  
    b2 = np.random.rand(48, 1) - 0.5
    W3 = np.random.rand(10, 48) - 0.5  
    b3 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))
    
def fwd_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

#BCWD PROP
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z>0
    
def back_prop(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y):
    m = Y.size
    ohY = one_hot(Y)
    dZ3 = A3 - ohY
    dW3 = (1/m) * dZ3.dot(A2.T)
    db3 = (1/m) * np.sum(dZ3)

    dZ2 = W3.T.dot(dZ3) * deriv_ReLU(Z2)
    dW2 = (1/m) * dZ2.dot(A1.T)
    db2 = (1/m) * np.sum(dZ2)
    
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = (1/m) * dZ1.dot(X.T)
    db1 = (1/m) * np.sum(dZ1)

    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - (alpha*dW1)
    b1 = b1 - (alpha*db1)
    W2 = W2 - (alpha*dW2)
    b2 = b2 - (alpha*db2)
    W3 = W3 - (alpha*dW3)
    b3 = b3 - (alpha*db3)
    return W1, b1, W2, b2, W3, b3

In [7]:
'''
def loss(X, Y, A2):
    m = Y.size
    log.likelihood = - np.log(A2[Y, range(m)])
    loss = (1/m) * np.sum(log.likelihood)
    return loss
'''
def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def grad_desc(X, Y, iterations, alpha):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations + 10):
        Z1, A1, Z2, A2, Z3, A3 = fwd_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = back_prop(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3= update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0:
            print("iteration: ", i)
            print("accuracy: ", get_accuracy(get_predictions(A3), Y))
    for iteration in range(iteration): # Exponential decay
        while(alpha > 0.001):
            alpha *= 0.95  
    return W1, b2, W2, b2, W3, b3

In [8]:
W1, b1, W2, b2, W3, b3 = grad_desc(X_train, Y_train, 1000, 0.1)

iteration:  0
[8 8 1 ... 7 8 8] [2 1 6 ... 4 0 0]
accuracy:  0.09475609756097561
iteration:  10
[5 1 6 ... 0 7 0] [2 1 6 ... 4 0 0]
accuracy:  0.4433170731707317
iteration:  20
[5 1 6 ... 0 7 0] [2 1 6 ... 4 0 0]
accuracy:  0.5721951219512195
iteration:  30
[2 1 6 ... 9 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.6526829268292683
iteration:  40
[2 1 6 ... 9 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.6994390243902439
iteration:  50
[2 1 6 ... 9 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.731780487804878
iteration:  60
[2 1 6 ... 9 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.7549756097560976
iteration:  70
[2 1 6 ... 4 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.773390243902439
iteration:  80
[2 1 6 ... 4 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.7875609756097561
iteration:  90
[2 1 6 ... 4 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.7993414634146342
iteration:  100
[2 1 6 ... 4 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.8097560975609757
iteration:  110
[2 1 6 ... 4 0 0] [2 1 6 ... 4 0 0]
accuracy:  0.8179268292682926
iteration:  120
[2 1 6 ... 4

KeyboardInterrupt: 